In [ ]:
gcloud beta dataproc clusters create cluster-ce8a --enable-component-gateway --bucket its_a_bukket --region us-central1 --subnet default --zone us-central1-b --master-machine-type n1-standard-2 --master-boot-disk-size 500 --num-workers 2 --worker-machine-type n1-standard-2 --worker-boot-disk-size 500 --image-version 1.3-deb9 --optional-components ANACONDA,JUPYTER --max-idle 7200s --scopes 'https://www.googleapis.com/auth/cloud-platform' --project sapient-origin-267302

Helpful links:

https://databricks-prod-cloudfront.cloud.databricks.com/public/4027ec902e239c93eaaa8714f173bcfc/3923635548890252/1357850364289680/4930913221861820/latest.html

https://spark.apache.org/docs/latest/ml-features.html#tf-idf



In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.enableHiveSupport().appName('reddit_pipeline').getOrCreate()
sc = spark.sparkContext

In [3]:
sc.version

'2.4.0-cdh6.3.0'

I will now load the data as a PySpark DataFrame, and confirm the load was successful by reviewing the first five rows of the text.

In [4]:
%time df_raw = spark.read.option("quote", "\"").option("escape", "\"").csv('politics_all_politics_all000000000000',header=True)

CPU times: user 2.82 ms, sys: 2.7 ms, total: 5.52 ms
Wall time: 8.28 s


In [7]:
df_raw.select("body").take(5)

[Row(body='almost a third of registered voters in CA are republican, though...'),
 Row(body="So, you're saying netanyahu wants peace?"),
 Row(body='But the Saudis said that the refugees are too dangerous to take in and have taken in 0...'),
 Row(body='Yep, "opt-out" has been changed to "you are permitted to ask"'),
 Row(body="They can answer however they'd like")]

In [22]:
from pyspark.sql.functions import col, when

df_raw.withColumn("body", when(col("body").isNull(), "sad so empty").otherwise(col("body")))

DataFrame[body: string, score_hidden: string, archived: string, name: string, author: string, author_flair_text: string, downs: string, created_utc: string, subreddit_id: string, link_id: string, parent_id: string, score: string, retrieved_on: string, controversiality: string, gilded: string, id: string, subreddit: string, ups: string, distinguished: string, author_flair_css_class: string, body_nopunct: string]

At present, I have yet to find an easy way to put words into lower case and remove non-letter values from a text string via the ``Pipeline``, so I will first clean the comment text column as a pre-processing step.

In [10]:
from pyspark.sql.functions import lower, regexp_replace

def make_lower(c):
    c = lower(c)

    return c


In [12]:
from pyspark.sql.functions import udf
lower_case_it = udf(lambda x: make_lower(x))

df_raw = df_raw.withColumn('body', lower_case_it('body')).collect()

Py4JJavaError: An error occurred while calling o139.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: 
Aborting TaskSet 5.0 because task 0 (partition 0)
cannot run anywhere due to node and executor blacklist.
Most recent failure:
Lost task 0.0 in stage 5.0 (TID 5, hd04.rcc.local, executor 6): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/cloudera/parcels/CDH/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in main
    process()
  File "/opt/cloudera/parcels/CDH/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 367, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/cloudera/parcels/CDH/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 342, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/opt/cloudera/parcels/CDH/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 141, in dump_stream

In [25]:
df

Py4JJavaError: An error occurred while calling o635.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: 
Aborting TaskSet 8.0 because task 0 (partition 0)
cannot run anywhere due to node and executor blacklist.
Most recent failure:
Lost task 0.1 in stage 8.0 (TID 47, hd04.rcc.local, executor 30): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/cloudera/parcels/CDH/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in main
    process()
  File "/opt/cloudera/parcels/CDH/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 367, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/cloudera/parcels/CDH/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 342, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/opt/cloudera/parcels/CDH/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 141, in dump_stre

In [4]:
# from pyspark.sql.functions import regexp_replace

# df_raw = df_raw.withColumn('body_nopunct', regexp_replace('body', "[^a-zA-Z0-9\\s]", ""))

In [ ]:
# %time df_raw.show(5)

In [5]:
df_raw.select("body_nopunct").show(10)

+--------------------+
|        body_nopunct|
+--------------------+
|almost a third of...|
|So youre saying n...|
|But the Saudis sa...|
|Yep optout has be...|
|They can answer h...|
|Theories have evi...|
|I think isnt cons...|
|Isnt consistent w...|
|As was stated els...|
|The US has 3 tier...|
+--------------------+
only showing top 10 rows



Here, we establish a custom Pipeline function to stem all tokens in a column using the NLTK Porter Stemmer.

In [26]:
from pyspark import keyword_only
import pyspark.sql.functions as F
from pyspark.sql import DataFrame
from pyspark.sql.types import ArrayType, StringType
from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, Param
#import nltk
from nltk.stem.porter import PorterStemmer

#this code courtesy of http://michael-harmon.com/blog/SentimentAnalysisP2.html

class PorterStemming(Transformer, HasInputCol, HasOutputCol):
    """
    PosterStemming class using the NLTK Porter Stemmer
    
    This comes from https://stackoverflow.com/questions/32331848/create-a-custom-transformer-in-pyspark-ml
    Adapted to work with the Porter Stemmer from NLTK.
    """
    
    @keyword_only
    def __init__(self, 
                 inputCol  : str = None, 
                 outputCol : str = None, 
                 min_size  : int = None):
        """
        Constructor takes in the input column name, output column name,
        plus the minimum legnth of a token (min_size)
        """
        # call Transformer classes constructor since were extending it.
        super(Transformer, self).__init__()

        # set Parameter objects minimum token size
        self.min_size = Param(self, "min_size", "")
        self._setDefault(min_size=0)

        # set the input keywork arguments
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

        # initialize Stemmer object
        self.stemmer  = PorterStemmer()

        
    @keyword_only
    def setParams(self, 
                  inputCol  : str = None, 
                  outputCol : str = None, 
                  min_size  : int = None
      ) -> None:
        """
        Function to set the keyword arguemnts
        """
        kwargs = self._input_kwargs
        return self._set(**kwargs)
    

    def _stem_func(self, words  : list) -> list:
        """
        Stemmer function call that performs stemming on a
        list of tokens in words and returns a list of tokens
        that have meet the minimum length requiremnt.
        """
        # We need a way to get min_size and cannot access it 
        # with self.min_size
        min_size       = self.getMinSize()

        # stem that actual tokens by applying 
        # self.stemmer.stem function to each token in 
        # the words list
        stemmed_words  = map(self.stemmer.stem, words)

        # now create the new list of tokens from
        # stemmed_words by filtering out those
        # that are not of legnth > min_size
        filtered_words = filter(lambda x: len(x) > min_size, stemmed_words)

        return list(filtered_words)
    
    def _transform(self, df: DataFrame) -> DataFrame:
        """
        Transform function is the method that is called in the 
        MLPipleline.  We have to override this function for our own use
        and have it call the _stem_func.

        Notice how it takes in a type DataFrame and returns type Dataframe
        """
        # Get the names of the input and output columns to use
        out_col       = self.getOutputCol()
        in_col        = self.getInputCol()

        # create the stemming function UDF by wrapping the stemmer 
        # method function
        stem_func_udf = F.udf(self._stem_func, ArrayType(StringType()))
        
        # now apply that UDF to the column in the dataframe to return
        # a new column that has the same list of words after being stemmed
        df2           = df.withColumn(out_col, stem_func_udf(df[in_col]))

        return df2
  
  
    def setMinSize(self,value):
        """
        This method sets the minimum size value
        for the _paramMap dictionary.
        """
        self._paramMap[self.min_size] = value
        return self

    def getMinSize(self) -> int:
        """
        This method uses the parent classes (Transformer)
        .getOrDefault method to get the minimum
        size of a token.
        """
        return self.getOrDefault(self.min_size)


With this custom function in place, I will build out a full pipeline that completes the following steps in order:

* Tokenize each comment string
* Remove stop words
* Stem each token
* Hashes the tokens
* Computes the IDF of those tokens

The pipeline will ultimately output a column named "features" that can be used for analysis.

In [27]:
from pyspark.ml.feature import HashingTF, Tokenizer, StopWordsRemover, IDF
from pyspark.ml import Pipeline

tokenizer = Tokenizer(inputCol="body_nopunct", outputCol="body_tokens")
remover = StopWordsRemover(inputCol = "body_tokens", outputCol = "body_tokens_nosw")
stemmer = PorterStemming(inputCol = "body_tokens_nosw", outputCol = "body_tokens_clean")
hashingTF = HashingTF(inputCol="body_tokens_clean", outputCol="raw_features")
#idf = IDF(inputCol="raw_features", outputCol="features")


In [28]:
pipeline = Pipeline(stages=[tokenizer, remover, stemmer, hashingTF])

In [29]:
%time test_fit = pipeline.fit(df_raw).transform(df_raw)


CPU times: user 45.1 ms, sys: 12.2 ms, total: 57.3 ms
Wall time: 160 ms


In [30]:
%time test_fit.take(1)

CPU times: user 5.21 ms, sys: 4.29 ms, total: 9.51 ms
Wall time: 3.65 s


[Row(body='almost a third of registered voters in CA are republican, though...', score_hidden=None, archived=None, name=None, author='Starkravingmad7', author_flair_text=None, downs=None, created_utc='1502391062', subreddit_id='t5_2cneq', link_id='t3_6stdi4', parent_id='t1_dlfk7kb', score='2', retrieved_on='1503941308', controversiality='0', gilded='0', id='dlft5kl', subreddit='politics', ups=None, distinguished=None, author_flair_css_class=None, body_nopunct='almost a third of registered voters in CA are republican though', body_tokens=['almost', 'a', 'third', 'of', 'registered', 'voters', 'in', 'ca', 'are', 'republican', 'though'], body_tokens_nosw=['almost', 'third', 'registered', 'voters', 'ca', 'republican', 'though'], body_tokens_clean=['almost', 'third', 'regist', 'voter', 'ca', 'republican', 'though'], raw_features=SparseVector(262144, {7761: 1.0, 12074: 1.0, 68641: 1.0, 74383: 1.0, 145449: 1.0, 197339: 1.0, 223821: 1.0}))]

In [31]:
idf = IDF(inputCol="raw_features", outputCol="features")

In [32]:
fitted_df= idf.fit(test_fit).transform(test_fit)

Py4JJavaError: An error occurred while calling o747.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 10.0 failed 4 times, most recent failure: Lost task 0.3 in stage 10.0 (TID 58, hd06.rcc.local, executor 32): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$createTransformFunc$1: (string) => array<string>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificMutableProjection.ScalaUDF_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificMutableProjection.apply(Unknown Source)
	at org.apache.spark.sql.execution.python.EvalPythonExec$$anonfun$doExecute$1$$anonfun$5.apply(EvalPythonExec.scala:124)
	at org.apache.spark.sql.execution.python.EvalPythonExec$$anonfun$doExecute$1$$anonfun$5.apply(EvalPythonExec.scala:122)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:410)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:410)
	at scala.collection.It

In [18]:
hash_TF = HashingTF(inputCol = "body_tokens_clean", outputCol = "raw_vecs")
tf = hash_TF.transform(test_fit)



In [19]:
# While applying HashingTF only needs a single pass to the data, applying IDF needs two passes:
# First to compute the IDF vector and second to scale the term frequencies by IDF.
tf.cache()
idf = IDF(inputCol = "raw_vecs", outputCol = "idf_vecs").fit(tf)
tfidf = idf.transform(tf)

Py4JJavaError: An error occurred while calling o476.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: 
Aborting TaskSet 7.0 because task 0 (partition 0)
cannot run anywhere due to node and executor blacklist.
Most recent failure:
Lost task 3.1 in stage 7.0 (TID 43, hd04.rcc.local, executor 28): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$createTransformFunc$1: (string) => array<string>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificMutableProjection.ScalaUDF_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificMutableProjection.apply(Unknown Source)
	at org.apache.spark.sql.execution.python.EvalPythonExec$$anonfun$doExecute$1$$anonfun$5.apply(EvalPythonExec.scala:124)
	at org.apache.spark.sql.execution.python.EvalPythonExec$$anonfun$doExecute$1$$anonfun$5.apply(EvalPythonExec.scala:122)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:410)
	at scala.collecti

In [ ]:
tfidf.take(1)